In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import logging
import pickle

import numpy as np

import py21cmfast as p21c

sys.path.append("..")
from dm21cm.dm_params import DMParams
from dm21cm.evolve import evolve
import dm21cm.physics as phys
from dm21cm.utils import load_dict

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps as cms
mpl.rc_file("../matplotlibrc")

/n/home07/yitians/.conda/envs/21cmfast/lib/python3.10/site-packages/py21cmfast/_cfg.py:58: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/n/home07/yitians/.conda/envs/21cmfast/lib/python3.10/site-packages/py21cmfast/_cfg.py:42: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")
INFO:root:Using 21cmFAST version 0.1.dev1579+g6b1da6d.d20230702


## config

In [2]:
print(os.cpu_count())

48


In [ ]:
# check cached runs
l = os.listdir(os.environ['P21C_CACHE_DIR'])
l.sort()
[print(i) for i in l];

In [ ]:
! ls -l /n/home07/yitians/dm21cm/DM21cm/data/run_info/DH

In [4]:
! mkdir /n/holyscratch01/iaifi_lab/yitians/21cmFAST-cache/phph_dhinit_s8zero_fine_debug_fine/

In [5]:
! cp /n/home07/yitians/dm21cm/DM21cm/data/run_info/DH/phph_debug_fine_soln.p /n/holyscratch01/iaifi_lab/yitians/21cmFAST-cache/phph_dhinit_s8zero_fine_debug_fine/dhinit_soln.p

In [10]:
import time
from tqdm import tqdm

In [11]:
for i in tqdm(range(100)):
    # sleep for 0.1 seconds
    time.sleep(0.1)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:10<00:00,  9.94it/s]


## run

In [14]:
dd = evolve(
    run_name = 'phph_dhinit_s8zero_fine_debug',
    run_mode = 'bath',
    z_start = 44,
    z_end = 6,
    zplusone_step_factor = 1.05,
    dm_params = DMParams(
        mode = 'swave',
        primary = 'phot_delta',
        m_DM = 1e10,
        sigmav = 1e-24,
    ),
    struct_boost_model = 'erfc 1e-3',
    enable_elec = False,
    dhinit_list = ['phot', 'T_k', 'x_e'],
    dhtf_version = 'debug_230802',
    
    p21c_initial_conditions = p21c.initial_conditions(
        user_params = p21c.UserParams(
            HII_DIM = 50, # [1] | base: 50
            BOX_LEN = 50, # [p-Mpc] | base: 50
            N_THREADS = 32
        ),
        cosmo_params = p21c.CosmoParams(
            OMm = 0.32,
            OMb = 0.049,
            POWER_INDEX = 0.96,
            SIGMA_8 = 1e-10, # base 0.83
            hlittle = 0.67
        ),
        random_seed = 54321,
        write = True,
    ),
    
    rerun_DH = False,
    clear_cache = True,
    force_reload_tf = True,
    use_tqdm = True,
    save_slices = True,
    debug = True,
    #debug_xe_func = interpolate.interp1d(soln['rs']-1, soln['x'][:,0]),
)

/n/home07/yitians/.conda/envs/21cmfast/lib/python3.10/site-packages/py21cmfast/inputs.py:487: UserWarning: The USE_INTERPOLATION_TABLES setting has changed in v3.1.2 to be default True. You can likely ignore this warning, but if you relied onhaving USE_INTERPOLATION_TABLES=False by *default*, please set it explicitly. To silence this warning, set it explicitly to True. Thiswarning will be removed in v4.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
rs 20.615018489612996
dE/dBavg_dt 5.575885e-18
struct_boost 235.7262879021981
dE/dBavg_dt 1.3143826e-15

In [ ]:
20.615018489619406 0.00131764251603822 5.4393989312728763e-14
hep.rs : 20.615018489619406
hep [eV/Bavg] : 2.9974439701319713
hed (ion exc heat cont) [eV/Bavg] : [0.         0.         0.00314856 0.        ]
eng inj rate un : 5.575676772884466e-18
struct boost : 235.72628790189134
eng inj rate : 1.3143335882128521e-15
dt 424059901545885.0
Tk : 0.012858813956977964

## debug

In [45]:
from darkhistory.spec.spectrum import Spectrum

In [49]:
from dm21cm.utils import load_dict

In [12]:
abscs = load_dict('../data/abscissas/abscs_debug_230802.h5')

In [40]:
dhinit_fn = f"{p21c.config['direc']}/dhinit_soln.p"

In [41]:
dhinit_fn

'/n/holyscratch01/iaifi_lab/yitians/21cmFAST-cache/phph_dhinit_s8zero_fine_debug/dhinit_soln.p'

In [42]:
dhinit_fn = f"{p21c.config['direc']}/dhinit_soln.p"
dhinit_soln = pickle.load(open(dhinit_fn, 'rb'))

In [43]:
dhinit_soln['rs'][340:350]

array([50.13214854, 49.53415788, 48.94330022, 48.35949048, 47.78264459,
       47.21267949, 46.64951309, 46.09306431, 45.543253  , 45.        ])

In [52]:
z_now = 41.857142857142854

logrs_dh_arr = np.log(dhinit_soln['rs'])[::-1]
logrs = np.log(1+z_now)
i = np.searchsorted(logrs_dh_arr, logrs)
logrs_left, logrs_right = logrs_dh_arr[i-1:i+1]

dh_spec_N_arr = np.array([s.N for s in dhinit_soln['highengphot']])[::-1]
dh_spec_left, dh_spec_right = dh_spec_N_arr[i-1:i+1]
dh_spec = ( dh_spec_left * np.abs(logrs - logrs_right) + \
            dh_spec_right * np.abs(logrs - logrs_left) ) / np.abs(logrs_right - logrs_left)
phot_bath_spec = Spectrum(abscs['photE'], dh_spec, rs=1+z_now, spec_type='N')
print(1+z_now, phot_bath_spec.toteng())

42.857142857142854 0.9256142780811808


In [53]:
np.dot(phot_bath_spec.N, abscs['photE'])

0.9256142780811807

In [26]:
dhinit_soln['highengphot'].rs

array([3000.        , 2964.21513859, 2928.85712927, 2893.92088045,
       2859.40136123, 2825.29360075, 2791.59268743, 2758.29376829,
       2725.39204821, 2692.88278929, 2660.76131015, 2629.02298524,
       2597.66324418, 2566.67757111, 2536.06150405, 2505.81063423,
       2475.92060548, 2446.38711356, 2417.20590562, 2388.37277951,
       2359.8835832 , 2331.73421421, 2303.92061897, 2276.43879228,
       2249.28477672, 2222.45466205, 2195.94458468, 2169.75072714,
       2143.86931745, 2118.29662864, 2093.02897821, 2068.06272757,
       2043.39428154, 2019.02008781, 1994.93663647, 1971.14045945,
       1947.62813006, 1924.39626248, 1901.4415113 , 1878.76057097,
       1856.35017542, 1834.2070975 , 1812.32814857, 1790.71017802,
       1769.35007284, 1748.24475712, 1727.39119167, 1706.78637354,
       1686.42733559, 1666.31114609, 1646.43490828, 1626.79575994,
       1607.390873  , 1588.21745313, 1569.27273931, 1550.55400348,
       1532.0585501 , 1513.7837158 , 1495.72686897, 1477.88540

In [ ]:
42.85714285715626 0.001658003439935829 5.4393989312728763e-14
hep.rs : 42.85714285715626
hep [eV/Bavg] : 0.8812251941902314
hed (ion exc heat cont) [eV/Bavg] : [0.         0.         0.00092562 0.        ]
rate_func_eng_unclustered(rs) : 9.907041531707055e-19
dt 141091198890861.44

In [67]:
for z, hepE, h, dt in zip(dd['z'], dd['photEtot'], dd['heat'], dd['dt']):
    print(1+z, hepE, h, dt)

42.857142857142854 0.9410799586942651 0.0009420371 141091198890927.88
40.81632653061224 0.9165066621399569 0.000917432 151842796881127.9
38.872691933916414 0.9001913530288917 0.0009010979 163411671514496.4
37.021611365634676 0.8946306798067117 0.0008955115 175859882361873.78
35.25867749108064 0.9028097472582385 0.00090373604 189254196880334.03
33.57969284864823 0.9281607241989236 0.0009290716 203666446695747.56
31.980659855855453 0.9744733394485955 0.0009754775 219173910643132.38
30.457771291290907 1.0458489622683715 0.0010469344 235859726524079.1
29.007401229800863 1.1465700430105594 0.0011476957 253813333671533.66
27.626096409334153 1.2809039282077075 0.0012821574 273130948548805.6
26.310568008889668 1.453100245181853 0.0014545958 293916075751210.0
25.05768381799016 1.6672405024264916 0.0016689051 316280056924285.5
23.864460779038247 1.92695623083765 0.0019288574 340342660260726.8
22.72805788479833 2.2355061568740293 0.0022377432 366232713387169.94
21.645769414093646 2.59569657210798

In [3]:
from dm21cm.evolve import get_z_edges, get_z_edges_old

In [ ]:
get_z_edges(44, 6, 1.01)

In [ ]:
get_z_edges_old(44, 6, 1.01)

In [ ]:
get_z_edges?